In [ ]:
import os
os.chdir("..")

In [ ]:
import polars as pl
from src.data.data_process import DataClean

dc = DataClean()

In [ ]:
df = pl.from_pandas(dc.dataset())

In [ ]:
data  = df.with_columns(
    iq=(pl.col("rule_of_law") + pl.col("control_of_corruption") + pl.col("regulatory_quality") + pl.col("political_stability") + pl.col("voice") + pl.col("government_effect")) / 6
)
IQ = data.select(pl.col("iq").median()).item()

data = data.group_by("country").agg(
    iq=pl.col("iq").mean()
)
data  = data.with_columns(
    iq_dummy = pl.when(pl.col("iq") >= IQ).then(1).otherwise(0)
)
dataset = df.join(data, on="country", how="left", validate="m:1")
dataset

In [ ]:
dataset.write_csv("dataset_iq.csv")
data.write_csv("iq_raw.csv")

In [ ]:
dc.wb_data(params=["RL.EST", "CC.EST", "RQ.EST", "PV.EST", "VA.EST", "GE.EST"], year=1996)